# ValyuContext

>[Valyu](https://www.valyu.network/) 允许 AI 应用程序和代理搜索互联网和专有数据源，以获取相关的 LLM 就绪信息。

本 Notebook 将介绍如何在 LangChain 中使用 Valyu 深度搜索工具。

首先，获取 Valyu API 密钥并将其添加为环境变量。通过[在此处注册](https://platform.valyu.network/)可获得 10 美元的免费积分。

## 设置

该集成位于 `langchain-valyu` 包中。

In [ ]:
%pip install -qU langchain-valyu

为了使用该包，您还需要将 `VALYU_API_KEY` 环境变量设置为您的 Valyu API 密钥。

In [ ]:
import os

valyu_api_key = os.environ["VALYU_API_KEY"]

## 实例化

现在我们可以实例化我们的检索器了：
`ValyuContextRetriever` 可以通过以下几个参数进行配置：

- `k: int = 5`  
  每次查询返回的 top 结果数量。

- `search_type: str = "all"`  
  要执行的搜索类型：'all'、'proprietary' 或 'web'。默认为 'all'。

- `relevance_threshold: float = 0.5`  
  文档被视为相关的最低相关性分数（介于 0 和 1 之间）。默认为 0.5。
  
- `max_price: float = 20.0`
  每次查询您愿意支付的最高价格（以美元计）。默认为 20.0。

- `start_date: Optional[str] = None`
  时间过滤的开始日期，格式为 YYYY-MM-DD（可选）。

- `end_date: Optional[str] = None`
  时间过滤的结束日期，格式为 YYYY-MM-DD（可选）。

- `client: Optional[Valyu] = None`  
  一个可选的自定义 Valyu 客户端实例。如果未提供，将内部创建一个新的客户端。
  
- `valyu_api_key: Optional[str] = None`  
  您的 Valyu API 密钥。如果未提供，检索器将查找 `VALYU_API_KEY` 环境变量。

In [ ]:
from langchain_valyu import ValyuRetriever

retriever = ValyuRetriever(
    k=5,
    search_type="all",
    relevance_threshold=0.5,
    max_price=20.0,
    start_date="2024-01-01",
    end_date="2024-12-31",
    client=None,
    valyu_api_key=os.environ["VALYU_API_KEY"],
)

## 用法

In [ ]:
query = "What are the benefits of renewable energy?"
docs = retriever.invoke(query)

for doc in docs:
    print(doc.page_content)
    print(doc.metadata)

## 在链中使用

我们可以轻松地将此检索器组合到链中。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)

llm = ChatOpenAI(model="gpt-4o-mini")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## API 参考

如需了解所有用于 Valyu Context API 功能和配置的详细文档，请前往 API 参考：https://docs.valyu.network/overview